# 第3章 サンプルコード

本ノートブックは「生成AIエージェント実践入門」第3章のサンプルコードです。

**環境セットアップについては、プロジェクトルートの `README.md` を参照してください（Python 3.12を使用します）。**

In [2]:
# 環境変数の読み込み
import os

# APIキーが設定されているか確認
if os.getenv("OPENAI_API_KEY") is None:
    print(
        "警告: OPENAI_API_KEYが設定されていません。プロジェクトルートのREADME.mdを参照し、環境設定を行ってください。"
    )

In [3]:
# 必要なライブラリのインポート
# このNotebookでは必要になった際に都度importします。

# OpenAI関連
from openai import OpenAI
# from openai import AzureOpenAI  # Azure OpenAIを使用する場合

# クライアントを定義
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# クライアントを定義（Azureの場合）
# client = AzureOpenAI(
#    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
#    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
# )

3.2 OpenAI  APIの基本

3.2.2 OpenAI APIの使い方

基本的なコード例

In [ ]:
# from openai import OpenAI

# クライアントを定義
# client = OpenAI(api_key="ここにAPIキーを入れます")

# Chat Completion APIの呼び出し例
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "こんにちは、今日はどんな天気ですか？"}],
)

# 応答内容を出力
print("Response:", response.choices[0].message.content)

消費されたトークン数の確認

In [ ]:
# 消費されたトークン数の表示
tokens_used = response.usage
print("Prompt Tokens:", tokens_used.prompt_tokens)
print("Completion Tokens:", tokens_used.completion_tokens)
print("Total Tokens:", tokens_used.total_tokens)
print("Completion_tokens_details:", tokens_used.completion_tokens_details)
print("Prompt_tokens_details:", tokens_used.prompt_tokens_details)

3.2.5 構造化出力（Structured Outputs）

jsonモードの設定例

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            "content": "あなたは JSON を出力するように設計された便利なアシスタントです。",
        },
        {"role": "assistant", "content": '{"winner": String}'},
        {"role": "user", "content": "2020 年のワールド シリーズの優勝者は誰ですか?"},
    ],
)

response.choices[0].message.content

# 出力例
# '{"year": 2020, "winner": "Los Angeles Dodgers"}'

Structured Outputsの実行例

In [ ]:
from pydantic import BaseModel


# Pydanticモデルを定義
class Recipe(BaseModel):
    name: str
    servings: int
    ingredients: list[str]
    steps: list[str]


# Structured Outputsに対応するPydanticモデルを指定して呼び出し
response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "トマトソースパスタのレシピを教えてください"}
    ],
    temperature=0,
    response_format=Recipe,
)
# 生成されたレシピ情報の表示
recipe = response.choices[0].message.parsed

print("Recipe Name:", recipe.name)
print("Servings:", recipe.servings)
print("Ingredients:", recipe.ingredients)
print("Steps:", recipe.steps)

3.3 Function callingの活用方法

3.3.1 Function callingの使い方

In [ ]:
import json


# 天気情報を取得するダミー関数
def get_weather(location):
    # 実際のAPI呼び出し部分を簡略化
    weather_info = {
        "Tokyo": "晴れ、気温25度",
        "Osaka": "曇り、気温22度",
        "Kyoto": "雨、気温18度",
    }
    return weather_info.get(location, "天気情報が見つかりません")


# 初回のユーザーメッセージ
messages = [{"role": "user", "content": "東京の天気を教えてください"}]

# モデルに提供するToolの定義
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "指定された場所の天気情報を取得します",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "都市名（例: Tokyo）",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

# モデルへの最初のAPIリクエスト
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
    tools=tools,
    tool_choice="auto",
)

# モデルの応答を処理
response_message = response.choices[0].message
messages.append(response_message)

print("モデルからの応答:")
print(response_message)

# 関数呼び出しを処理
if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        if tool_call.function.name == "get_weather":
            function_args = json.loads(tool_call.function.arguments)
            print(f"関数の引数: {function_args}")
            weather_response = get_weather(location=function_args.get("location"))
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_weather",
                    "content": weather_response,
                }
            )
else:
    print("モデルによるツール呼び出しはありませんでした。")

# モデルへの最終的なAPIリクエスト
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    temperature=0,
)

print("Final Response:", final_response.choices[0].message.content)

3.4 生成AIエージェントで利用されるTool

3.4.1 WEB検索

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3), tavily_api_key=os.getenv("TAVILY_API_KEY")]

In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel


# 引数スキーマを定義
class AddArgs(BaseModel):
    a: int
    b: int


@tool(args_schema=AddArgs)
def add(a: int, b: int) -> int:
    """
    このToolは2つの整数を引数として受け取り、それらの合計を返します。

    Args:
        a (int): 加算する最初の整数。
        b (int): 加算する2つ目の整数。

    Returns:
        int: 2つの整数の合計値。

    使用例:
        例:
            入力: {"a": 3, "b": 5}
            出力: 8
    """
    return a + b


# 実行例
args = {"a": 5, "b": 10}
result = add.func(**args)  # Toolを呼び出す
print(f"Result: {result}")  # Result: 15

# Toolに関連付けられている属性の確認
print(add.name)
print(add.description)
print(add.args)

LangChainを使ったDuckduckgoのカスタムツールの例

In [10]:
from duckduckgo_search import DDGS
from pydantic import BaseModel, Field
from itertools import islice


class DDGSearchInput(BaseModel):
    """検索クエリが文字列であることをバリデーションします。
    文字列以外のデータ型の検索入力を受け付けません。
    """

    query: str = Field(description="検索キーワードを入力してください")


@tool(args_schema=DDGSearchInput)
def duckduckgo_search(query: str, max_result_num: int = 5) -> list[dict[str, str]]:
    """
    このToolはDuckDuckGoを使用してWeb検索を実行します。

    機能:
        このToolは指定されたキーワード（query）でDuckDuckGo検索を行い、
        検索結果から指定した数（max_result_num）までの結果を取得します。
        各検索結果にはタイトル、スニペット、およびURLが含まれます。

    Args:
        query (str): 検索キーワード。
        max_result_num (int): 取得する検索結果の最大数。デフォルトは5。

    Returns:
        List[Dict[str, str]]: 検索結果のリスト。各要素は以下の形式の辞書です:
            - "title" (str): 検索結果のタイトル。
            - "snippet" (str): 検索結果のスニペット（概要）。
            - "url" (str): 検索結果のURL。
    """
    with DDGS() as ddgs:
        responce = ddgs.text(query, region="jp-jp", safesearch="off", backend="lite")
        return [
            {
                "title": r.get("title", ""),
                "snippet": r.get("body", ""),
                "url": r.get("href", ""),
            }
            for r in islice(responce, max_result_num)
        ]

In [ ]:
# HTML取得
import requests

# DuckDuckGo検索を実行
search_query = "AIエージェント 実践本"
search_results = duckduckgo_search.func(query=search_query, max_result_num=3)

# 検索結果を表示
print("\n検索結果:")
for i, result in enumerate(search_results):
    print(f"\n{i + 1}. {result['title']}")
    print(f"   概要: {result['snippet'][:100]}...")
    print(f"   URL: {result['url']}")

# 最初の検索結果のURLを取得
if search_results:
    url = search_results[0]["url"]
    print(f"\n最初の検索結果のURLにアクセスしています: {url}")

    # Webページを取得
    try:
        response = requests.get(url)
        html_content = response.content
        print(f"\nHTTPステータスコード: {response.status_code}")
        print(f"\nHTMLコンテンツの大きさ: {len(html_content)} bytes")
        print(f"\nHTMLコンテンツの最初の部分: \n{html_content[:500]}...")
    except Exception as e:
        print(f"\nエラーが発生しました: {e}")
else:
    print("\n検索結果はありませんでした")

SQLDatabaseChainの使用例

In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


# 引数スキーマを定義
class SQLQueryArgs(BaseModel):
    keywords: str


@tool(args_schema=SQLQueryArgs)
def text_to_sql_search(keywords: str):
    """
    自然言語でのクエリをSQLクエリに変換し、SQLデータベースで検索を実行します。

    機能:
        - このToolは、与えられた自然言語形式のキーワードをもとに、SQLクエリを生成します。
        - LLMを使用してSQL文を生成し、PostgreSQLデータベースで検索を実行します。
        - 取得した検索結果を返します。

    Args:
        keywords (str): 実行したいクエリの自然言語キーワード。
            例: "2023年の売上が最大の月を教えてください"

    Returns:
        Any: データベース検索結果を返します。
    """

    # PostgreSQLデータベース接続パラメータを設定する
    db_url = f"postgresql+psycopg2://{os.getenv('PGUSER')}:{os.getenv('PGPASSWORD')}@{os.getenv('PGHOST')}:5432/{os.getenv('PGDATABASE')}"
    db = SQLDatabase.from_uri(db_url)

    llm = ChatOpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="gpt-4o",
        temperature=0.0,
    )

    # SQLチェーンの設定
    db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

    # 実行
    response = db_chain.run(keywords)

    return response


# 実行例
args = {"keywords": "employeeテーブルの情報は何件ありますか？"}
response = text_to_sql_search.func(**args)
print(response)

3.7 LangGraphによるエージェントワークフロー構築

3.7.2 エージェントワークフローの構築方法

1. 状態（State）とワークフローの初期化

In [13]:
# LangGraphでエージェントのワークフローの初期化
from typing import TypedDict
from langgraph.graph import END, StateGraph, START


# ワークフロー前端の状態を記録するためのクラス
# 基本的に各ノードにこのクラスが引数に渡される
class AgentState(TypedDict):
    input: str  # ユーザの入力
    plans: list[str]  # 計画ノードの結果
    feedbacks: list[str]  # 振り返りノードの結果
    output: str  # 生成ノードの結果
    iteration: int


# Graph全体を定義
workflow = StateGraph(AgentState)

2. ノードとエッジの設定

In [14]:
# LangGraphでエージェントワークフローの構築


# 各ノードの処理、エッジでの条件判定関数を定義
def plan_node(state: AgentState) -> AgentState:
    # 現在の入力に基づいて計画を作成
    plan = f"ブログ記事「{state['input']}」の作成計画:"
    plans = state.get("plans", [])
    plans.append(
        plan
        + "\n1. イントロダクション\n2. LangGraphの基本概念\n3. シンプルなワークフロー例\n4. まとめ"
    )

    # 状態を更新して返す
    return {**state, "plans": plans}


def generation_node(state: AgentState) -> AgentState:
    # 計画に基づいて出力を生成
    iteration = state["iteration"]
    # イテレーション数を増やす
    iteration += 1

    # 現在の計画を取得
    plan = state["plans"][-1] if state["plans"] else "計画なし"

    # 出力を生成
    output = f"イテレーション {iteration} の出力:\n"
    if iteration == 1:
        output += "# LangGraphを用いたエージェントワークフロー構築方法\n\n## はじめに\nLangGraphは、大規模言語モデル(LLM)を使用したエージェントやワークフローを構築するためのフレームワークです。"
    elif iteration == 2:
        output += "## LangGraphの基本概念\n\n1. **状態（State）**: ワークフロー全体で共有される情報\n2. **ノード（Node）**: 処理を行う関数\n3. **エッジ（Edge）**: ノード間の接続と遷移条件"
    elif iteration == 3:
        output += "## LangGraphの実装例\n\n```python\nimport operator\nfrom typing import TypedDict, List\nfrom langgraph.graph import END, StateGraph, START\n\nclass AgentState(TypedDict):\n    input: str\n    output: str\n```"
    else:
        output += "## まとめ\n\nLangGraphを使うことで、複雑なエージェントの振る舞いを制御しやすくなります。状態管理とワークフローの分離により、メンテナンス性の高いAIアプリケーションが開発可能です。"

    # 状態を更新して返す
    return {**state, "output": output, "iteration": iteration}


def reflection_node(state: AgentState) -> AgentState:
    # 現在の出力を振り返り、フィードバックを生成
    output = state["output"]
    feedbacks = state.get("feedbacks", [])

    # フィードバックを生成
    feedback = f"フィードバック (イテレーション {state['iteration']}):\n"
    if state["iteration"] == 1:
        feedback += "イントロダクションは良いですが、もう少し具体的な例やメリットを追加すると良いでしょう。"
    elif state["iteration"] == 2:
        feedback += (
            "基本概念の説明は分かりやすいです。次はコード例を加えると良いでしょう。"
        )
    elif state["iteration"] == 3:
        feedback += "コード例が示されていますが、もう少し詳しい説明や実行結果があると良いでしょう。"

    feedbacks.append(feedback)

    # 状態を更新して返す
    return {**state, "feedbacks": feedbacks}


# 使用するノードを追加。ノード名と対応する関数を書く。名前はこの後も使うので一意である必要がある
workflow.add_node("planner", plan_node)
workflow.add_node("generator", generation_node)
workflow.add_node("reflector", reflection_node)

# エントリーポイントを定義。これが最初に呼ばれるノード
workflow.add_edge(START, "planner")


# 条件付きエッジ用の条件。3回イテレーションする
def should_continue(state: AgentState):
    if state["iteration"] > 3:  # iterationは整数なのでlen()を使わない
        # End after 3 iterations
        return END
    return "reflector"


# ノードをつなぐエッジを追加
workflow.add_edge("planner", "generator")
workflow.add_conditional_edges("generator", should_continue, ["reflector", END])
workflow.add_edge("reflector", "generator")


# 最後にworkflowをコンパイルする。これでLangChainのrunnnableな形式になる
# runnnableになることでinvokeやstreamが使用できるようになる
app = workflow.compile()

3. 実行

In [ ]:
# エージェントのワークフローの実行
inputs = {
    "input": "LangGraphを用いたエージェントワークフロー構築方法のブログ記事を作成して",
    "iteration": 0,  # iterationの初期値を設定
    "plans": [],  # plansの初期値も設定
    "feedbacks": [],  # feedbacksの初期値も設定
    "output": "",  # outputの初期値も設定
}

for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

In [ ]:
from IPython.display import Image, display

# mermaidで描画
display(Image(app.get_graph().draw_mermaid_png()))